In [28]:
import pandas as pd
import math
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import numpy as np
from sklearn.cross_validation import KFold
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint as sp_randint
from time import time
from operator import itemgetter
from sklearn.grid_search import RandomizedSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from operator import itemgetter
import re
from sklearn.preprocessing import LabelEncoder

In [29]:
train_data=pd.read_csv('counterfeit_train.csv')
test_data=pd.read_csv('counterfeit_test.csv')

In [30]:
train_data.head()

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402


In [31]:
for col in train_data.columns:
    print(col,':',train_data[col].nunique())

Medicine_ID : 1557
Counterfeit_Weight : 414
DistArea_ID : 10
Active_Since : 9
Medicine_MRP : 5097
Medicine_Type : 16
SidEffect_Level : 2
Availability_rating : 6317
Area_Type : 4
Area_City_Type : 3
Area_dist_level : 4
Counterfeit_Sales : 3142


In [32]:
f = lambda x: x.fillna(0 if x.isnull().all() else x.value_counts().index[0])

In [33]:
train_data['Counterfeit_Weight'] = (train_data.groupby('Medicine_ID')['Counterfeit_Weight'].transform(f))
test_data['Counterfeit_Weight'] = (train_data.groupby('Medicine_ID')['Counterfeit_Weight'].transform(f))                             

In [34]:
for col in ["DistArea_ID"]:
    train_data[col]=train_data[col].astype("str")
    test_data[col]=test_data[col].astype("str")
    train_data[col]=[re.sub('[A-Za-z]', '', x) for x in train_data[col]]
    test_data[col]=[re.sub('[A-Za-z]', '', x) for x in test_data[col]]

In [35]:
train_data['DistArea_ID']=pd.to_numeric(train_data['DistArea_ID'],errors="coerce")
test_data['DistArea_ID']=pd.to_numeric(test_data['DistArea_ID'],errors="coerce")

In [10]:
# le=LabelEncoder()

In [11]:
# le.fit(train_data['Medicine_Type'].values)
# train_data['Medicine_Type']= le.transform(train_data['Medicine_Type'].values)
# test_data['Medicine_Type']= le.transform(test_data['Medicine_Type'].values)

In [36]:
for col in ['SidEffect_Level','Area_Type','Area_City_Type','Area_dist_level']:
    temp=pd.get_dummies(train_data[col],prefix=col,drop_first=True)
    train_data=pd.concat([temp,train_data],1)
    train_data.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(test_data[col],prefix=col,drop_first=True)
    test_data=pd.concat([temp,test_data],1)
    test_data.drop([col],1,inplace=True)

In [37]:
train_data.head()

,Area_dist_level_Medium,Area_dist_level_Small,Area_dist_level_Unknown,Area_City_Type_Tier 2,Area_City_Type_Tier 3,Area_Type_DownTown,Area_Type_Industrial,Area_Type_MidTownResidential,SidEffect_Level_mild,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,Availability_rating,Counterfeit_Sales
0,0,1,0,0,0,1,0,0,0,RRA15,13.100,46,1995,160.2366,Antimalarial,0.070422,1775.5026
1,1,0,0,0,1,0,0,0,1,YVV26,7.450,27,1983,110.4384,Mstablizers,0.013000,3069.1520
2,0,1,0,0,0,1,0,0,1,LJC15,9.025,46,1995,259.4092,Cardiac,0.060783,2603.0920
3,0,1,0,0,0,1,0,0,1,GWC40,11.800,46,1995,99.9830,OralContraceptives,0.065555,1101.7130
4,0,1,0,0,0,0,0,1,0,QMN13,8.930,19,1983,56.4402,Hreplacements,0.248859,158.9402


In [38]:
train_data['Medicine_Type'].unique()

array(['Antimalarial', 'Mstablizers', 'Cardiac', 'OralContraceptives',
       'Hreplacements', 'Antiseptics', 'Statins', 'Analgesics',
       'MuscleRelaxants', 'Antibiotics', 'Antifungal', 'Antiviral',
       'Tranquilizers', 'Antacids', 'Antipyretics', 'Stimulants'], dtype=object)

In [39]:
test_data['Medicine_Type'].unique()

array(['Antibiotics', 'OralContraceptives', 'Antipyretics',
       'Tranquilizers', 'Antacids', 'Cardiac', 'Hreplacements',
       'Mstablizers', 'Antiseptics', 'Antifungal', 'MuscleRelaxants',
       'Analgesics', 'Statins', 'Antimalarial', 'Stimulants', 'Antiviral'], dtype=object)

In [40]:
round(train_data.groupby("Medicine_Type")["Counterfeit_Sales"].mean())

Medicine_Type
Analgesics            2091.0
Antacids              2260.0
Antibiotics           2397.0
Antifungal            2444.0
Antimalarial          2193.0
Antipyretics          2293.0
Antiseptics           2367.0
Antiviral             2583.0
Cardiac               2363.0
Hreplacements         2376.0
Mstablizers           2045.0
MuscleRelaxants       2091.0
OralContraceptives    2245.0
Statins               2309.0
Stimulants            2280.0
Tranquilizers         2084.0
Name: Counterfeit_Sales, dtype: float64

In [41]:
temp=pd.get_dummies(train_data["Medicine_Type"],prefix="Medicine_Type",drop_first=True)
train_data=pd.concat([temp,train_data],1)
train_data.drop(["Medicine_Type"],1,inplace=True)
    
temp=pd.get_dummies(test_data["Medicine_Type"],prefix="Medicine_Type",drop_first=True)
test_data=pd.concat([temp,test_data],1)
test_data.drop(["Medicine_Type"],1,inplace=True)

In [42]:
x = train_data.drop('Counterfeit_Sales', 1)
y = train_data['Counterfeit_Sales']

In [43]:
x.drop('Medicine_ID', 1, inplace=True)

In [44]:
n_iter_search = 50
param_dist = {"n_estimators":[10,100,500,700],
              "max_depth": [3,5, None],
              "max_features": sp_randint(5, 11),
              "min_samples_split": sp_randint(5, 11),
              "min_samples_leaf": sp_randint(5, 11),
              "bootstrap": [True, False],
              }

In [45]:
clf = RandomForestRegressor(verbose=1,n_jobs=-1)
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

In [46]:
random_search.fit(x,y)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapse

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=1, warm_start=False),
          fit_params={}, iid=True, n_iter=50, n_jobs=1,
          param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x055048D0>, 'bootstrap': [True, False], 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x05504430>, 'n_estimators': [10, 100, 500, 700], 'max_depth': [3, 5, None], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x05504710>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=0)

In [47]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    #above line selects top n grid scores
    #for loop below , prints the rank, score and parameter combination
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")
 


In [48]:
report(random_search.grid_scores_)

Model with rank: 1
Mean validation score: 0.595 (std: 0.012)
Parameters: {'bootstrap': True, 'max_features': 9, 'min_samples_split': 6, 'n_estimators': 500, 'max_depth': None, 'min_samples_leaf': 8}

Model with rank: 2
Mean validation score: 0.594 (std: 0.012)
Parameters: {'bootstrap': True, 'max_features': 10, 'min_samples_split': 9, 'n_estimators': 500, 'max_depth': None, 'min_samples_leaf': 7}

Model with rank: 3
Mean validation score: 0.593 (std: 0.012)
Parameters: {'bootstrap': True, 'max_features': 8, 'min_samples_split': 8, 'n_estimators': 700, 'max_depth': None, 'min_samples_leaf': 8}



In [49]:
rf=RandomForestRegressor(n_estimators=500,verbose=1,criterion='mse',min_samples_split=6,
                         bootstrap=True,max_depth=None,max_features=9,min_samples_leaf=8,
                          )

In [50]:
rf.fit(x,y)

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.4s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=9, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=8,
           min_samples_split=6, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
           verbose=1, warm_start=False)

In [51]:
prediction=rf.predict(test_data.drop(['Medicine_ID'],1))


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.0s finished


In [52]:
submission=pd.DataFrame(list(zip(test_data['Medicine_ID'],list(prediction))),
                       columns=['Medicine_ID','Counterfeit_Sales'])

In [53]:
submission.to_csv('submission.csv',index=False)